In [0]:
#  Mount Google drive for storing intermediate results
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

**Data Collection**

The dataset is taken from Tensorflow CNN-Dailymail Dataset.

In [2]:
# using CNN-dailymail dataset available in tensorflow datasets
import tensorflow as tf
import tensorflow_datasets as tfds

data, info = tfds.load('cnn_dailymail', with_info=True)
dataset_train, dataset_test = data['train'], data['test']

#combining the highlights to create summary
def map_fn(x, start=tf.constant('<start>'), end=tf.constant('<end>')):
    strings = [start, x['highlights'], end]
    x['highlights'] = tf.strings.join(strings, separator=' ')
    return x

dataset_train = dataset_train.map(map_fn) 
elem,  = dataset_train.take(1)

Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteY8JN1L/cnn_dailymail-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteY8JN1L/cnn_dailymail-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteY8JN1L/cnn_dailymail-test.tfrecord


Dataset cnn_dailymail downloaded and prepared to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0. Subsequent calls will reuse this data.


In [6]:
print("------------------------------------Sample article-------------------------------------")
#printing subset of article to show how data looks
print(str(elem['article'].numpy()[0:1000]))
print("------------------------------------Sample summary-------------------------------------")
print(elem['highlights'].numpy())



------------------------------------Sample article-------------------------------------
b'(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as "The Milk Man," of using his role as a police officer to help the drug trafficking organization in exchange for money and gifts, including a Rolex watch. In one instance, the complaint alleges, Mata arranged to pay two assassins to kill rival drug dealers. The killers would pose as cops, pulling over their targets before shooting them, according to the complaint. "Ultimately, the (organization) decided not to move forward with the murder plot, but Mata still received a p

**Preprocessing** 

The code below expands the contractions, removes punctuations and removes stopwords.

In [3]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#list of contractions to be expanded
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

def clean_text(text, remove_stopwords = True):
  # print(text)
  if True:
    text = text.split()
    new_text = []
    for word in text:
        if word in contractions:
            new_text.append(contractions[word])
        else:
            new_text.append(word)
    text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%,.!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\"', ' ', text)
    text = re.sub('cnn', '', text)
    text = re.sub('<start>', '', text)
    text = re.sub('<end>', '', text)
  

    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
    

    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Example of Data Preprocessing** 

In [4]:
!pip3 install unidecode

     |████████████████████████████████| 245kB 36.2MB/s 


In [7]:
from unidecode import unidecode
#get single instance of data
elem = list(dataset_train.take(5))[1]

#data before preprocessing (showing subset of article)
print("---------------------------- data before processing --------------")
mystr = str(elem['article'].numpy(),'utf-8')
print(mystr[0 : 899])

#data after preprocessing (showing subset of article)
print("---------------------------- data after processing --------------")
print(clean_text(unidecode(mystr.lower()))[0 : 790])

---------------------------- data before processing --------------
(CNN) -- Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops. Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns. A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as "The Milk Man," of using his role as a police officer to help the drug trafficking organization in exchange for money and gifts, including a Rolex watch. In one instance, the complaint alleges, Mata arranged to pay two assassins to kill rival drug dealers. The killers would pose as cops, pulling over their targets before shooting them, according to the complaint. "Ultimately, the (organization) decided not to move forward with the murder plot, but Mata still r
---------------------------- data

**Storing Clean Data**

Preprocess the entire data and store the clean results in JSON file. This JSON file will be further used for word embedding creation and training of model.

In [0]:
import json
from unidecode import unidecode

#selecting 60,000 articles and summaries from dataset
elements = list( dataset_train.take(60000))
article_highlight_dict = {}
article_id = 1

for elem in elements: 
  updated_article = str(elem['article'].numpy(), 'utf-8')
  updated_article = unidecode(updated_article)
  updated_article = updated_article[1:]
  updated_higlight = str(elem['highlights'].numpy(), 'utf-8')
  updated_higlight = unidecode(updated_higlight)
  article_highlight_dict[article_id] = {'article': clean_text(updated_article.lower())}
  article_highlight_dict[article_id]['highlight'] =  clean_text(updated_higlight.lower(), False)
  article_id += 1

output = open('/gdrive/My Drive/project/article_highlight_Sample.json' , 'w')
json.dump(article_highlight_dict, output)
output.close()

